In [1]:

import pandas as pd

df = pd.read_csv(r"C:\Users\fabri\Desktop\MarcoPatierno_DepositoCorsoPython\env\Giorno 20 30-04\esercitazione\AB_NYC_2019.csv")
df.head(4)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194


In [2]:
df.info()
df.drop(columns=['id', 'name', 'host_name', 'last_review'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [3]:
df.isna().sum()

host_id                               0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [4]:
df["reviews_per_month"].fillna(0, inplace=True)

C:\Users\fabri\AppData\Local\Temp\ipykernel_14644\1901275697.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["reviews_per_month"].fillna(0, inplace=True)


In [5]:
df

,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
3,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48890,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,0.00,2,9
48891,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,0.00,2,36
48892,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,0.00,1,27
48893,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,0.00,6,2


In [6]:
df["neighbourhood"].unique()
df["neighbourhood"].value_counts()

neighbourhood
Williamsburg          3920
Bedford-Stuyvesant    3714
Harlem                2658
Bushwick              2465
Upper West Side       1971
                      ... 
Richmondtown             1
Fort Wadsworth           1
New Dorp                 1
Rossville                1
Willowbrook              1
Name: count, Length: 221, dtype: int64

In [7]:
!pip install category_encoders


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

# df contiene le colonne 'neighbourhood' e 'price'
kf = KFold(n_splits=5, shuffle=True, random_state=42)
te = TargetEncoder(smoothing=20)  

# Prepara la Series vuota
X_encoded = pd.Series(index=df.index, dtype=float)
global_mean = df['price'].mean()

for train_idx, valid_idx in kf.split(df):
    # 1) fit sull'insieme di train
    te.fit(df.loc[train_idx, ['neighbourhood']], df.loc[train_idx, 'price'])
    # 2) trasformi il validation set e ottieni una Serie
    enc_df = te.transform(df.loc[valid_idx, ['neighbourhood']])
    # 'neighbourhood' è il nome della colonna prodotta dal trasform
    # e può variare a seconda della versione di category_encoders
    # quindi verifica il nome con enc_df.columns
    col = enc_df.columns[0]  
    enc_values = enc_df[col].values  
    # 3) assegna per posizione (iloc) i valori nella Series
    X_encoded.iloc[valid_idx] = enc_values

# 4) riempi eventuali NaN (categorie non viste nei fold) con la media globale
X_encoded.fillna(global_mean, inplace=True)

# Aggiungi la colonna al tuo DataFrame
df['nbhd_te'] = X_encoded


In [9]:
df.drop(columns=['neighbourhood'], inplace=True)

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# definisci il transformer con dtype=float
cat_transformer = OneHotEncoder(handle_unknown='ignore', dtype=float)

preprocessor = ColumnTransformer(transformers=[
    ('ohe', cat_transformer, ['neighbourhood_group','room_type'])
], remainder='passthrough')

In [11]:
# One-hot encode neighbourhood_group
neighbourhood_group_dummies = pd.get_dummies(df['neighbourhood_group'], prefix='neighbourhood_group')

# One-hot encode room_type
room_type_dummies = pd.get_dummies(df['room_type'], prefix='room_type')

# Concatenate with the original dataframe
df = pd.concat([df, neighbourhood_group_dummies, room_type_dummies], axis=1)

# Drop the original categorical columns
df.drop(columns=['neighbourhood_group', 'room_type'], inplace=True)

# Display the first few rows to verify
df.head()

,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,nbhd_te,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,2787,40.64749,-73.97237,149,1,9,0.21,6,365,90.570561,False,True,False,False,False,False,True,False
1,2845,40.75362,-73.98377,225,1,45,0.38,2,355,277.916201,False,False,True,False,False,True,False,False
2,4632,40.80902,-73.94190,150,3,0,0.00,1,365,115.763219,False,False,True,False,False,False,True,False
3,4869,40.68514,-73.95976,89,1,270,4.64,1,194,189.224138,False,True,False,False,False,True,False,False
4,7192,40.79851,-73.94399,80,10,9,0.10,1,0,135.230159,False,False,True,False,False,True,False,False


Root Mean Squared Error: 202.29115387851243
R^2 Score: 0.07496976852416992
